## DHS Data Analysis

This notebook calculates label statistics for the DHS survey labels and generates a CSV file of SDG coverage by country.

## Pre-requisites

Go through the [`3_process_tfrecords.ipynb`](./3_process_tfrecords.ipynb) notebook.

## Imports and Constants

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
DHS_FINAL_CSV_PATH = 'output_labels/dhs_final_labels.csv'

## DHS Label Statistics

In [ ]:
# load final DHS CSV
label_cols = ['asset_index', 'water_index', 'sanitation_index', 'under5_mort', 'women_edu', 'women_bmi']
obs_cols = ['n_asset', 'n_water', 'n_sanitation', 'n_under5_mort', 'n_women_edu', 'n_women_bmi']
dtypes = {label: np.float64 for label in label_cols} | {n: 'Int64' for n in obs_cols}

dhs_final = pd.read_csv(DHS_FINAL_CSV_PATH, dtype=dtypes)
dhs_final.set_index('DHSID_EA', verify_integrity=True, inplace=True)
dhs_final['survey'] = dhs_final.index.str[:10]
display(dhs_final.head())

In [ ]:
# calculate # of observations, clusters, and countries per label
label_stats = pd.DataFrame({
    '# of observations':
        dhs_final[obs_cols].sum().rename(dict(zip(obs_cols, label_cols))),
    '# clusters':
        dhs_final[label_cols].notna().sum(),
    '# countries': pd.Series({
        label: dhs_final.loc[dhs_final[label].notna(), 'cname'].nunique()
        for label in label_cols
    })
})
display(label_stats)
print(label_stats)

In [ ]:
api_url = 'https://api.dhsprogram.com/rest/dhs/surveys?returnFields=SurveyId,SurveyType,CountryName&f=csv'
all_surveys = pd.read_csv(api_url)
assert len(all_surveys) == all_surveys['SurveyId'].nunique()
all_surveys['cname'] = all_surveys['SurveyId'].str[:2]
all_surveys['year'] = all_surveys['SurveyId'].str[2:6].astype(int)
display(all_surveys.head())

In [ ]:
included_surveys = dhs_final[['cname', 'year', 'survey']].drop_duplicates().reset_index(drop=True)
merged = included_surveys.merge(all_surveys, how='left', on=['cname', 'year'], validate='1:m')
print('# of surveys:', merged['survey'].nunique())

In [ ]:
print(merged.sort_values('year').groupby(['cname', 'CountryName']).agg(list)['SurveyId'].to_csv())

In [ ]:
# Some countries may have had multiple surveys in the same year, such as
# Uganda 2011, which had both UG2011DHS and UG2011AIS. The DHSID_EA column
# identifies the two surveys as:
# - UG2011DHS = UG-2011-6#
#   - the '#' indicates this is a numerically-versioned survey. On the DHS
#     website (https://dhsprogram.com/data/dataset/Uganda_Standard-DHS_2011.cfm)
#     we see that UG2011DHS files are named UG{DD}61{FF}, where "61" indicates
#     DHS Round 6, Version 1. This is what we mean by a "numerically-versioned"
#     survey.
# - UG2011AIS = UG-2011-6Z
#   - the 'Z' indicates this is a alphabetically-versioned survey. On the DHS
#     website (https://dhsprogram.com/data/dataset/Uganda_Standard-AIS_2011.cfm)
#     we see that UG2011AIS files are named UG{DD}6A{FF}, where "6A" indicates
#     DHS Round 6, Version A. This is what we mean by an "alphabetically-versioned"
#     survey.
merged.groupby(['cname', 'year']).filter(lambda df: len(df) > 1)

In [ ]:
# number of processed npzs
num_processed = !find dhs_npzs -type f | wc -l
num_processed = int(num_processed[0])

# number of final clusters to include
num_final = len(dhs_final)

print('num processed npzs:', num_processed)
print('num final clusters:', num_final)
print('diff (should match "num npzs missing labels" from log printout):', num_processed - num_final)

## SDG Coverage by Country

Outputs a CSV that shows which SDG is covered in which country, based on DHS surveys

|  | DHS_CountryCode | ISO2_CountryCode | ISO3_CountryCode | CountryName  | 1_pov | 3_health | 4_edu | 6_water
|--|-----------------|------------------|------------------|--------------|-------|----------|-------|----------
|0 | AL              | AL               | ALB              | Albania      | True  | True     | True  | True
|1 | AM              | AM               | ARM              | Armenia      | True  | True     | True  | True
|2 | AO              | AO               | AGO              | Angola       | True  | True     | True  | True
|3 | BD              | BD               | BGD              | Bangladesh   | True  | True     | True  | True
|4 | BF              | BF               | BFA              | Burkina Faso | True  | True     | True  | True

In [ ]:
df = pd.read_csv(DHS_FINAL_CSV_PATH)
display(df.head())

sdgs = pd.DataFrame(
    data=False, dtype=bool,
    index=pd.Index(df['cname'].unique(), name='dhs_cc'),
    columns=['1_pov', '3_health', '4_edu', '6_waterb'])
display(sdgs.head())

In [ ]:
pov_countries = df.loc[df['asset_index'].notna(), 'cname'].unique()
health_countries = df.loc[df['women_bmi'].notna() | df['under5_mort'].notna(), 'cname'].unique()
edu_countries = df.loc[df['women_edu'].notna(), 'cname'].unique()
water_countries = df.loc[df['water_index'].notna() | df['sanitation_index'].notna(), 'cname'].unique()

sdgs.loc[pov_countries, '1_pov'] = True
sdgs.loc[health_countries, '3_health'] = True
sdgs.loc[edu_countries, '4_edu'] = True
sdgs.loc[water_countries, '6_water'] = True

In [ ]:
dhs_countries_api = "http://api.dhsprogram.com/rest/dhs/countries?returnFields=CountryName,DHS_CountryCode,ISO2_countryCode,ISO3_countryCode&f=csv"
dhs_countries_crosswalk = pd.read_csv(dhs_countries_api)
display(dhs_countries_crosswalk.head())

In [ ]:
final = sdgs.merge(dhs_countries_crosswalk, how='inner', left_on='dhs_cc', right_on='DHS_CountryCode', validate='1:1')
final = final[['DHS_CountryCode', 'ISO2_CountryCode', 'ISO3_CountryCode', 'CountryName', '1_pov', '3_health', '4_edu', '6_water']]
display(final.head())

In [ ]:
final.to_csv('sdgs_coverage_by_country.csv', index=False)